In [1]:
pip install slideio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 MB 6.4 MB/s eta 0:00:00


In [2]:
pip install wget

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9674 sha256=dacc1766a4a3f2f646c0b1def2e5bd67426d2e5d40cdcb5a043113a792d88dd1
  Stored in directory: /root/.cache/pip/wheels/bd/a8/c3/3cf2c14a1837a4e04bd98631724e81f33f462d86a1d895fae0
Successfully built wget


In [3]:
!apt-get install openslide-tools
!apt-get install python-openslide
!pip install openslide-python

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  libopenslide0
Suggested packages:
  libtiff-tools
The following NEW packages will be installed:
  libopenslide0 openslide-tools
0 upgraded, 2 newly installed, 0 to remove and 21 not upgraded.
Need to get 92.5 kB of archives.
After this operation, 268 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libopenslide0 amd64 3.4.1+dfsg-2 [79.8 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 openslide-tools amd64 3.4.1+dfsg-2 [12.7 kB]
Fetched 92.5 kB in 1s (170 kB/s)
Selecting previously unselected package libopenslide0.
(Reading database ... 124016 files and directories currently installed.)
Preparing to unpack .../libopenslide0_3.4.1+dfsg-2

In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [5]:
import requests
import slideio
import wget
import matplotlib.pyplot as plt
from PIL import Image
import os
from openslide import open_slide
import openslide
from PIL import Image
import numpy as np

In [6]:
# api-endpoint
URL = 'https://api.gdc.cancer.gov/files?filters={"op":"and","content":[{"op":"=","content":{"field":"files.data_format","value":["SVS"]}},{"op":"=","content":{"field":"cases.project.project_id","value":["TCGA-LUAD"]}},{"op":"=","content":{"field":"data_type","value":["Slide Image"]}}]}&fields=file_id,associated_entities.entity_submitter_id&pretty=TRUE&from=0&size=99999'
URL = URL.replace(" ", "%20")

# sending get request and saving the response as response object
r = requests.get(url = URL)
  
# extracting data in json format
myfile = r.json()

In [7]:
print(myfile)

{'data': {'hits': [{'id': 'bed9aa12-1018-41e7-aed3-96f899c46aa0', 'associated_entities': [{'entity_submitter_id': 'TCGA-78-7156-01A-01-TS1'}], 'file_id': 'bed9aa12-1018-41e7-aed3-96f899c46aa0'}, {'id': '3abc179d-0c8a-4f5f-b1d3-fd1e7b5bf3f1', 'associated_entities': [{'entity_submitter_id': 'TCGA-78-7156-01A-01-BS1'}], 'file_id': '3abc179d-0c8a-4f5f-b1d3-fd1e7b5bf3f1'}, {'id': '4e6c4c7a-085d-480d-ae27-8b91c88ebe8a', 'associated_entities': [{'entity_submitter_id': 'TCGA-73-7498-01A-01-TS1'}], 'file_id': '4e6c4c7a-085d-480d-ae27-8b91c88ebe8a'}, {'id': '304fc295-f208-40b6-8d66-b5cef921ade0', 'associated_entities': [{'entity_submitter_id': 'TCGA-73-7498-01Z-00-DX1'}], 'file_id': '304fc295-f208-40b6-8d66-b5cef921ade0'}, {'id': 'bdf2c904-7f95-412f-bc36-5136692c165c', 'associated_entities': [{'entity_submitter_id': 'TCGA-44-A47G-01A-02-TSB'}], 'file_id': 'bdf2c904-7f95-412f-bc36-5136692c165c'}, {'id': 'e3c73aec-e169-43f0-a454-a898015bc39e', 'associated_entities': [{'entity_submitter_id': 'TCGA-

In [8]:
print(myfile.keys())

dict_keys(['data', 'warnings'])


In [9]:
print(myfile['data'])

{'hits': [{'id': 'bed9aa12-1018-41e7-aed3-96f899c46aa0', 'associated_entities': [{'entity_submitter_id': 'TCGA-78-7156-01A-01-TS1'}], 'file_id': 'bed9aa12-1018-41e7-aed3-96f899c46aa0'}, {'id': '3abc179d-0c8a-4f5f-b1d3-fd1e7b5bf3f1', 'associated_entities': [{'entity_submitter_id': 'TCGA-78-7156-01A-01-BS1'}], 'file_id': '3abc179d-0c8a-4f5f-b1d3-fd1e7b5bf3f1'}, {'id': '4e6c4c7a-085d-480d-ae27-8b91c88ebe8a', 'associated_entities': [{'entity_submitter_id': 'TCGA-73-7498-01A-01-TS1'}], 'file_id': '4e6c4c7a-085d-480d-ae27-8b91c88ebe8a'}, {'id': '304fc295-f208-40b6-8d66-b5cef921ade0', 'associated_entities': [{'entity_submitter_id': 'TCGA-73-7498-01Z-00-DX1'}], 'file_id': '304fc295-f208-40b6-8d66-b5cef921ade0'}, {'id': 'bdf2c904-7f95-412f-bc36-5136692c165c', 'associated_entities': [{'entity_submitter_id': 'TCGA-44-A47G-01A-02-TSB'}], 'file_id': 'bdf2c904-7f95-412f-bc36-5136692c165c'}, {'id': 'e3c73aec-e169-43f0-a454-a898015bc39e', 'associated_entities': [{'entity_submitter_id': 'TCGA-73-4659-0

In [10]:
print(len(myfile['data']['hits']))

1608


In [11]:
file_id = myfile['data']['hits'][1]['file_id']
print(file_id)

3abc179d-0c8a-4f5f-b1d3-fd1e7b5bf3f1


## Resmi İndirme

In [ ]:
path = '/content/gdrive/MyDrive/TCGA-LUAD/'

for i in range(126, len(myfile['data']['hits'])):

  file_id = myfile['data']['hits'][i]['file_id']

  URL = "https://api.gdc.cancer.gov/data/{}".format(file_id)

  file_name = wget.download(URL, out = path)

  print('Image Successfully Downloaded: ', file_name)


Image Successfully Downloaded:  /content/gdrive/MyDrive/TCGA-LUAD//TCGA-35-3615-01A-01-BS1.40ee8b17-cbd1-4b88-bfd9-0685f18f3829.svs
Image Successfully Downloaded:  /content/gdrive/MyDrive/TCGA-LUAD//TCGA-MP-A4SV-01Z-00-DX1.5430F2CF-BE4B-42D6-9CD4-92E6FB3F6E99.svs
Image Successfully Downloaded:  /content/gdrive/MyDrive/TCGA-LUAD//TCGA-MP-A4TC-01Z-00-DX1.C7DB4A7F-F6B8-4FA0-84C1-9B87D41C7749.svs
Image Successfully Downloaded:  /content/gdrive/MyDrive/TCGA-LUAD//TCGA-86-8280-01A-01-TS1.03ddd678-176c-4608-89e6-4f8ee6de5cc4.svs
Image Successfully Downloaded:  /content/gdrive/MyDrive/TCGA-LUAD//TCGA-44-3396-01A-01-BS1.020af312-29f3-4506-941a-6e74f0f3212c.svs
Image Successfully Downloaded:  /content/gdrive/MyDrive/TCGA-LUAD//TCGA-53-A4EZ-01Z-00-DX1.5D155F0B-A677-4589-AF00-A4C451F5B6B6.svs
Image Successfully Downloaded:  /content/gdrive/MyDrive/TCGA-LUAD//TCGA-38-A44F-01A-01-TSA.48E36C45-F2C3-4BE6-84F5-F65FF6AA4962.svs
Image Successfully Downloaded:  /content/gdrive/MyDrive/TCGA-LUAD//TCGA-L9-A

## Resmi dönüştürme

In [ ]:
s_path = '/content/gdrive/MyDrive/TCGA/TCGA-LUAD/'

In [ ]:
svs_list = [svs_file for svs_file in os.listdir(s_path) if svs_file.endswith(".svs")]

for s in svs_list:

  try:

    #Load the slide file (svs) into an object.
    slide = open_slide(s_path + s)
    
    slide_props = slide.properties
    #print(slide_props)
    
    #print("Pixel size of X in um is:", slide_props['openslide.mpp-x'])
    #print("Pixel size of Y in um is:", slide_props['openslide.mpp-y'])

    slide_dims = slide.dimensions
    #print(slide_dims[0], slide_dims[1])
    #print(min(slide_dims[0], slide_dims[1]))

    """
    #Get a thumbnail of the image and visualize
    slide_thumb_600 = slide.get_thumbnail(size=(150, 150))
    slide_thumb_600.show()

    #Convert thumbnail to numpy array
    slide_thumb_600_np = np.array(slide_thumb_600)
    plt.figure(figsize=(8,8))
    # plt.imshow(slide_thumb_600_np) 
    """

    dims = slide.level_dimensions

    num_levels = len(dims)
    #print("Number of levels in this image are:", num_levels)

    #print("Dimensions of various levels in this image are:", dims)

    #By how much are levels downsampled from the original image?
    factors = slide.level_downsamples
    #print("Each level is downsampled by an amount of: ", factors)

    SCALE_FACTOR = 600
    best_level = slide.get_best_level_for_downsample(SCALE_FACTOR)

    #Copy an image from a level
    level4_dim = dims[best_level]
    
    #Give pixel coordinates (top left pixel in the original large image)
    #Also give the level number (for level 3 we are providing a valueof 2)
    #Size of your output image
    #Remember that the output would be a RGBA image (Not, RGB)
    level3_img = slide.read_region((0,0), best_level, level4_dim) #Pillow object, mode=RGBA
    level3_img.show()

    #print(level3_img.size)

    newsize = (150, 150)
    level3_img = level3_img.resize(newsize)

    #print(level3_img.size)

    #Convert the image to RGB
    level3_img_RGB = level3_img.convert('RGB')
    level3_img_RGB.show()

    #Convert the image into numpy array for processing
    level3_img_np = np.array(level3_img_RGB)
    #plt.imshow(level3_img_np)
    
    plt.imsave('/content/gdrive/MyDrive/TCGA-LUAD-150/' + s[:-4] + ".tiff", level3_img_np)
    print("saved " + s)

  except:
    continue


## Not

In [ ]:
    """
    #Generate object for tiles using the DeepZoomGenerator
    tiles = DeepZoomGenerator(slide, tile_size=256, overlap=0, limit_bounds=False)
    #Here, we have divided our svs into tiles of size 256 with no overlap. 

    #The tiles object also contains data at many levels. 
    #To check the number of levels
    print("The number of levels in the tiles object are: ", tiles.level_count)

    print("The dimensions of data in each level are: ", tiles.level_dimensions)

    #Total number of tiles in the tiles object
    print("Total number of tiles = : ", tiles.tile_count)

    #How many tiles at a specific level?
    level_num = 11
    print("Tiles shape at level ", level_num, " is: ", tiles.level_tiles[level_num])
    print("This means there are ", tiles.level_tiles[level_num][0]*tiles.level_tiles[level_num][1], " total tiles in this level")

    #Dimensions of the tile (tile size) for a specific tile from a specific layer
    tile_dims = tiles.get_tile_dimensions(11, (3,4)) #Provide deep zoom level and address (column, row)


    #Tile count at the highest resolution level (level 16 in our tiles)
    tile_count_in_large_image = tiles.level_tiles[16] #126 x 151 (32001/256 = 126 with no overlap pixels)
    #Check tile size for some random tile
    tile_dims = tiles.get_tile_dimensions(16, (120,140))
    #Last tiles may not have full 256x256 dimensions as our large image is not exactly divisible by 256
    tile_dims = tiles.get_tile_dimensions(16, (125,150))


    single_tile = tiles.get_tile(16, (62, 70)) #Provide deep zoom level and address (column, row)
    single_tile_RGB = single_tile.convert('RGB')
    single_tile_RGB.show()

    ###### Saving each tile to local directory
    cols, rows = tiles.level_tiles[16]

    import os
    tile_dir = "images/saved_tiles/original_tiles/"
    for row in range(rows):
        for col in range(cols):
            tile_name = os.path.join(tile_dir, '%d_%d' % (col, row))
            print("Now saving tile with title: ", tile_name)
            temp_tile = tiles.get_tile(16, (col, row))
            temp_tile_RGB = temp_tile.convert('RGB')
            temp_tile_np = np.array(temp_tile_RGB)
            plt.imsave(tile_name + ".png", temp_tile_np)
    """

https://towardsdatascience.com/slideio-a-new-python-library-for-reading-medical-images-11858a522059



## Svs to Tiff

In [ ]:
import os, sys, pdb
import numpy as np
from scipy import misc
import matplotlib.pyplot as plt
from skimage import io
import openslide
import argparse

In [ ]:
def batch_convert_svs(slide_dir, tile_size=8192):
  svs_list = [svs_file for svs_file in os.listdir(slide_dir) if svs_file.endswith(".svs")]
  for ind, svs_file in enumerate(svs_list):
    print("Processing {}/{}: {}".format(ind+1, len(svs_list), svs_file))
    save_svs_img(svs_file, tile_size)

In [ ]:
def save_svs_img(slide_filename, tile_size=8192):
	slide_file = openslide.OpenSlide(slide_filename)
	slide_width, slide_height = slide_file.dimensions

	# tile_arr = []
	slide_img = np.zeros((slide_height, slide_width, 3), np.uint8)	
	x_tile_num = int(np.floor((slide_width-1)/tile_size)) + 1
	y_tile_num = int(np.floor((slide_height-1)/tile_size)) + 1
	for iy in range(y_tile_num):	
		for ix in range(x_tile_num):
			start_x = ix * tile_size
			len_x = tile_size if (ix + 1) * tile_size < slide_width else (slide_width - start_x) 
			start_y = iy * tile_size
			len_y = tile_size if (iy + 1) * tile_size < slide_height else (slide_height - start_y)
			# tile_arr.append(((start_x, start_y), (len_x, len_y)))
			cur_tile = slide_file.read_region(location=(start_x, start_y), level=0, size=(len_x, len_y))
			slide_img[start_y:start_y+len_y, start_x:start_x+len_x, :] = np.array(cur_tile)[:,:,:3]

	slide_savename = os.path.splitext(slide_filename)[0] + '.tif'
	# misc.imsave(slide_savename, slide_img)
	io.imsave(slide_savename, slide_img)

In [ ]:
source_files = "image"
tile_size = "8192"

batch_convert_svs(source_files, tile_size)

Processing 1/1: TCGA-C8-A27A-01Z-00-DX1.0E26C46D-CD65-40F3-8976-EB4415582934.svs


OpenSlideUnsupportedFormatError: ignored